In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import regularizers, optimizers
from keras.layers import Input,Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
import secrets
from PIL import Image

cwd = os.getcwd()
data_dir = cwd+ '/Data for test'  
rand_seed = secrets.randbelow(1_000_000_000) # random seed for train/val split
batch_size = 32
input_size = (64, 64)

# data augmentation by rescaling, randomly applying shearing transformations and zooming, randomly horizonal flips
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2) # set validation split

train_data = train_datagen.flow_from_directory(  
    data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='sparse',  # 1d integer labels
    subset='training') # set as training data
    # classes= None - list of classes are automatically inferred from subdirectory names

val_data = train_datagen.flow_from_directory(
    data_dir, # same directory as training data
    target_size=input_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation') # set as validation data

# dictionary that maps the class labels to the index
label_map = (train_data.class_indices)
label_dict = {val: key for key, val in label_map.items()}
print(label_dict)

Found 240 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
{0: 'Bed', 1: 'Chair', 2: 'Sofa'}


In [2]:
### construct the cnn model with regularization (L2 regularizer and 20% droput) 
cnn = keras.Sequential(
    [
        Input((64,64, 3)),
        Conv2D(16, (3, 3), activation='relu',strides=(1, 1),padding='valid',
                     kernel_regularizer=regularizers.L2(0.01)),
        Conv2D(16, (3, 3), activation='relu',strides=(1, 1),padding='valid',
                      kernel_regularizer=regularizers.L2(0.01)), 
        MaxPooling2D((2, 2),strides=(2, 2),padding='valid'),
        Dropout(0.2),  

        Conv2D(24, (3, 3), activation='relu',strides=(1, 1),padding='valid',
                     kernel_regularizer=regularizers.L2(0.01)), 
        Conv2D(24, (3, 3), activation='relu',strides=(1, 1),padding='valid',
                     kernel_regularizer=regularizers.L2(0.01)),         
        MaxPooling2D((2, 2),strides=(2, 2),padding='valid'),   
        Dropout(0.2),  
        
        Flatten(),
        Dense(3, activation='softmax')   
    ], 
)

# compile the model
cnn.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=['accuracy'])

# train the model
cnn.fit(train_data, validation_data=(val_data),
                      epochs=50, batch_size=batch_size, shuffle=True)  

# save the model for use in api
cnn.save('cnn_model')

Epoch 1/50
8/8 [==============================] - 17s 2s/step - loss: 1.7311 - accuracy: 0.3208 - val_loss: 1.6659 - val_accuracy: 0.3667
Epoch 2/50
8/8 [==============================] - 15s 2s/step - loss: 1.6034 - accuracy: 0.5750 - val_loss: 1.5100 - val_accuracy: 0.7333
Epoch 3/50
8/8 [==============================] - 14s 2s/step - loss: 1.4228 - accuracy: 0.6000 - val_loss: 1.2000 - val_accuracy: 0.7333
Epoch 4/50
8/8 [==============================] - 20s 3s/step - loss: 1.0648 - accuracy: 0.7083 - val_loss: 0.7797 - val_accuracy: 0.9000
Epoch 5/50
8/8 [==============================] - 21s 3s/step - loss: 0.9247 - accuracy: 0.7875 - val_loss: 0.7886 - val_accuracy: 0.8000
Epoch 6/50
8/8 [==============================] - 13s 2s/step - loss: 0.8104 - accuracy: 0.8000 - val_loss: 0.7103 - val_accuracy: 0.9000
Epoch 7/50
8/8 [==============================] - 17s 2s/step - loss: 0.7933 - accuracy: 0.8125 - val_loss: 0.5979 - val_accuracy: 0.8833
Epoch 8/50
8/8 [==================

INFO:tensorflow:Assets written to: cnn_model\assets
